In [9]:
import pandas as pd
import mysql.connector


In [10]:
file_path = r"C:\Users\deepak\OneDrive\Desktop\secure check 1\police_logs.csv"
df = pd.read_csv(file_path)
df.head()


C:\Users\deepak\AppData\Local\Temp\ipykernel_34496\1131777066.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,0:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,0:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,0:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,0:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,0:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305


In [19]:
import mysql.connector

df['search_type'] = df['search_type'].fillna("None")
 
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="deepak@2005",
    database="securecheck"
)
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS traffic_stops;")
conn.commit()
print("🗑️ Old table dropped!")


🗑️ Old table dropped!


In [20]:
create_table_query = """
CREATE TABLE IF NOT EXISTS traffic_stops (
    id INT AUTO_INCREMENT PRIMARY KEY,
    stop_date DATE,
    stop_time TIME,
    country_name VARCHAR(100),
    driver_gender VARCHAR(10),
    driver_age_raw INT,
    driver_age INT,
    driver_race VARCHAR(50),
    violation_raw VARCHAR(255),
    violation VARCHAR(255),
    search_conducted BOOLEAN,
    search_type VARCHAR(255),
    stop_outcome VARCHAR(255)
);
"""
cursor.execute(create_table_query)
print("✅ Table ready in database!")


✅ Table ready in database!


In [21]:
df = df.where(pd.notnull(df), None)


In [22]:
# Replace NaN with None (so MySQL can accept it)
df = df.where(pd.notnull(df), None)

# Insert each row into MySQL
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO traffic_stops (
            stop_date, stop_time, country_name, driver_gender, driver_age_raw,
            driver_age, driver_race, violation_raw, violation,
            search_conducted, search_type, stop_outcome
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['stop_date'], row['stop_time'], row['country_name'], row['driver_gender'],
        row['driver_age_raw'], row['driver_age'], row['driver_race'],
        row['violation_raw'], row['violation'], row['search_conducted'],
        row['search_type'], row['stop_outcome']
    ))

conn.commit()
print("✅ All CSV data inserted successfully!")

cursor.close()
conn.close()


✅ All CSV data inserted successfully!


In [23]:
df.isnull().sum()


stop_date             0
stop_time             0
country_name          0
driver_gender         0
driver_age_raw        0
driver_age            0
driver_race           0
violation_raw         0
violation             0
search_conducted      0
search_type           0
stop_outcome          0
is_arrested           0
stop_duration         0
drugs_related_stop    0
vehicle_number        0
dtype: int64